Practical Work in AI - Concept Models
Tragler Thomas
====================


In [1]:
from importlib import reload
import pandas as pd
import os
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

import derm7pt_data
from derm7pt_data import Derm7pt_data


reload(derm7pt_data)

path = os.path.normpath('Data\\Derm7pt')

derm7pt = Derm7pt_data(path)
metadata = derm7pt.metadata
print(metadata.shape, len(derm7pt.images))

(1011, 19) 2013


In [2]:
#Data loading
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)
my_dataset = metadata

#split data into training, validation and test sets
train_size = int(0.8 * len(my_dataset))
val_size = int(0.15 * len(my_dataset))
test_size = len(my_dataset) - train_size - val_size
#ToDo stratified sampling
#overall weird spliting ToDo
#train_split, val_split, test_split = random_split(my_dataset, [train_size, val_size, test_size])

#train_data, train_concepts, train_label = derm7pt.data_split_by_index(train_split.indices)
#val_data, val_concepts, val_label = derm7pt.data_split_by_index(val_split.indices)
#test_data, test_concepts, test_label = derm7pt.data_split_by_index(test_split.indices)

#Dataload
train_loader = DataLoader(derm7pt, batch_size=32, shuffle=False)


device:  cuda


In [3]:
my_dataset.columns

Index(['case_num', 'diagnosis', 'seven_point_score', 'pigment_network',
       'streaks', 'pigmentation', 'regression_structures', 'dots_and_globules',
       'blue_whitish_veil', 'vascular_structures',
       'level_of_diagnostic_difficulty', 'elevation', 'location', 'sex',
       'clinic', 'derm', 'nums', 'abbrevs', 'info'],
      dtype='object')

In [4]:
#Torch CNN model with 3 Conv layers and 3 fully connected layers
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #conv Layers
        in_channels, out_channels = (3, 16)
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        # in_channels, out_channels = (out_channels, 2*out_channels)
        # self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        # in_channels, out_channels = (out_channels, 2*out_channels)
        # self.conv3 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        
        #Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=4, stride=4, padding=0)
        
        self.channels_before_linear = out_channels
        # Fully connected layers
        self.fc1 = nn.Linear(self.channels_before_linear * 192 * 128, 128)  # Adjust the input size based on your image dimensions
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)  # Output layer with one neuron for binary classification
        
        self.relu = nn.ReLU()

    def forward(self, x):
        # Convolutional layers with activation and pooling
        x = self.relu(self.conv1(x))
        # x = self.relu(self.conv2(x))
        # x = self.relu(self.conv3(x))
        
        #Pooling layer
        x = self.pool(x)

        # Flatten the output for fully connected layers
        x = x.view(-1, self.channels_before_linear * 192 * 128)  # Corrected input size based on spatial dimensions

        # Fully connected layers with activation
        x = self.relu(self.fc1(x))
        # x = self.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    

In [5]:
#Training the model
import torch.optim as optim

# Instantiate the model
model = Net()
model.to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 3
for epoch in range(n_epochs):
    running_loss = 0.0
    for i, batch in enumerate(train_loader, 0):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass, backward pass, and optimization
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # Print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
    #break
print('Finished Training')

[1,    10] loss: 17.351
[1,    20] loss: -38.905
[1,    30] loss: -386.171
[2,    10] loss: 155.553
[2,    20] loss: -1266.914
[2,    30] loss: -4679.586
[3,    10] loss: 1306.549
[3,    20] loss: -8919.701
[3,    30] loss: -25973.174
Finished Training


In [6]:
#TODO Labels are not binary